In [6]:
!pip uninstall -y numpy
!pip install numpy==1.23.5  # 또는 1.24.3 정도로 낮춰 설치


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 103.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
xarray 2025.3.1 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
bigframes 2.4.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
blosc2 3.3.2 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires num

In [1]:
!pip install -q qai-hub onnx numpy==1.26.4

import qai_hub as hub
import onnx
import numpy as np
import os
from PIL import Image


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 103.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,

In [2]:
# ✅ 본인 API 토큰 입력
!qai-hub configure --api_token 42f67116ca75d4c803448ee68598707d3861ab6a


qai-hub configuration saved to /root/.qai_hub/client.ini
==================== /root/.qai_hub/client.ini ====================
[api]
api_token = 42f67116ca75d4c803448ee68598707d3861ab6a
api_url = https://app.aihub.qualcomm.com
web_url = https://app.aihub.qualcomm.com
verbose = True




In [4]:
device = hub.Device("RB3 Gen 2 (Proxy)")

compile_job = hub.submit_compile_job(
    model="yolov8n.onnx",
    device=device,
    input_specs={"images": (1, 3, 640, 640)},
    options="--target_runtime onnx"
)
optimized_model = compile_job.get_target_model()
print("✅ ONNX compile 완료")


Uploading yolov8n.onnx


100%|██████████| 11.7M/11.7M [00:00<00:00, 36.0MB/s]


Scheduled compile job (jgzoly8o5) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jgzoly8o5/

Waiting for compile job (jgzoly8o5) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          
✅ ONNX compile 완료


In [6]:
def load_calibration_images(folder_path, input_shape=(1, 3, 640, 640)):
    mean = np.array([0.485, 0.456, 0.406]).reshape((1, 3, 1, 1))  # NCHW
    std = np.array([0.229, 0.224, 0.225]).reshape((1, 3, 1, 1))
    images = []

    for fname in sorted(os.listdir(folder_path))[:20]:
        path = os.path.join(folder_path, fname)
        img = Image.open(path).convert("RGB").resize((640, 640))
        img_np = np.array(img).astype(np.float32) / 255.0  # HWC, 0~1
        img_np = np.transpose(img_np, (2, 0, 1))  # CHW
        img_np = np.expand_dims(img_np, axis=0)  # NCHW
        img_np = (img_np - mean) / std
        images.append(img_np.astype(np.float32))

    return images

calibration_data = {"images": load_calibration_images("calib_images")}
print(f"✅ Calibration 이미지 {len(calibration_data['images'])}장 로드 완료")


✅ Calibration 이미지 10장 로드 완료


In [7]:
quant_job = hub.submit_quantize_job(
    model=optimized_model,  # 앞에서 compile 완료된 ONNX 모델
    calibration_data=calibration_data,  # 지금 만든 실사 이미지 기반
    weights_dtype=hub.QuantizeDtype.INT8,
    activations_dtype=hub.QuantizeDtype.INT8,
)

quantized_model = quant_job.get_target_model()
print("✅ 양자화 완료")


Uploading dataset: 12.5MB [00:00, 37.4MB/s]                            


Scheduled quantize job (j5mq0kodp) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/j5mq0kodp/

Waiting for quantize job (j5mq0kodp) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          
✅ 양자화 완료


In [8]:
tflite_job = hub.submit_compile_job(
    model=quantized_model,
    device=device,
    options="--target_runtime tflite --quantize_io"
)

tflite_model = tflite_job.get_target_model()
tflite_model.download("wheelChair_YOLOv8-Detection-Quantized.tflite")
print("✅ 최종 TFLite 모델 다운로드 완료")


Scheduled compile job (jp20rd4rg) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp20rd4rg/

Waiting for compile job (jp20rd4rg) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          


wheelChair_YOLOv8-Detection-Quantized.tflite: 100%|██████████| 3.10M/3.10M [00:00<00:00, 44.0MB/s]

Downloaded model to wheelChair_YOLOv8-Detection-Quantized.tflite
✅ 최종 TFLite 모델 다운로드 완료


In [19]:
# 예시: test_image.jpg를 추론용 input으로 준비
from PIL import Image
import numpy as np

image_path = "calib_images/pexels-elevate-3009792.jpg"
image = Image.open(image_path).convert("RGB").resize((640, 640))
img_np = np.array(image).astype(np.float32) / 255.0
img_np = np.transpose(img_np, (2, 0, 1))  # CHW
img_np = np.expand_dims(img_np, axis=0)  # NCHW

input_tensor = img_np.astype(np.float32)


In [20]:
inference_job = hub.submit_inference_job(
    model=tflite_model,  # 또는 quantized_model도 가능
    device=device,       # RB3 Gen 2 또는 Galaxy S24 등
    inputs={"images": [input_tensor]}
)

# 결과 다운로드
output = inference_job.download_output_data()
print("✅ 추론 완료, 출력 키:", list(output.keys()))


Uploading dataset: 1.54MB [00:00, 8.39MB/s]                   


Scheduled inference job (jpee7xlop) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jpee7xlop/

Waiting for inference job (jpee7xlop) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          


tmp6c0200tj.h5: 100%|██████████| 14.4k/14.4k [00:00<00:00, 12.5MB/s]

✅ 추론 완료, 출력 키: ['output_0']
